In [1]:
from z3 import *
from __future__ import annotations
from dataclasses import dataclass

In [2]:
import sys
# change the path below to match your setup
sys.path.append('/home/mfredrik/316-livecode/lab1-2023/src')

The following are useful imports from the starter code

In [3]:
from parser import parse, fmla_parse, term_parse
from tinyscript_util import (
    stringify,
    term_enc,
    fmla_enc
)
import tinyscript as tn

Here's a quick demonstration of the parser and pretty-printer. The program on display is one of the test cases, to give an idea of what they look like.

In [4]:
swap = "x := x + y; y := x - y; x := x - y"
swap_alpha = parse(swap)
print(stringify(swap_alpha))

x := (x)+(y);
y := (x)-(y);
x := (x)-(y)


## Implementing the box axioms

Below is our partial implementation of the box modality. The remaining cases are left as part of the lab.

In [5]:
# Apply axioms of dynamic logic for [alpha] P
def box(alpha : tn.Prog, P : z3.BoolRef) -> z3.BoolRef:
	match alpha:
		case tn.Asgn(x, e):
			return z3.substitute(P, [(term_enc(tn.Var(x)), term_enc(e))])
		case tn.Seq(alpha1, alpha2):
			return box(alpha1, box(alpha2, P))

A basic sanity check: `[x := 1](x < 0)`

We should get `1 < 0` (or something equivalent), which simplifies to `False`

In [6]:
alpha = tn.Asgn('x', tn.Const(1))
post = tn.LtF(tn.Var('x'), tn.Const(0))
pre = box(alpha, fmla_enc(post))
print('Program:', stringify(alpha))
print('Verification condition:', pre)

Program: x := 1
Verification condition: Not(0 <= 1)


In [7]:
pre = fmla_enc(fmla_parse("x == a && y == b"))
post = fmla_enc(fmla_parse("x == b && y == a"))
vc = z3.Implies(pre, box(swap_alpha, post))
vc

Implies(And(x == a, y == b),
        And(x + y + -1*(x + y + -1*y) == b,
            x + y + -1*y == a))

Now we can use Z3 to get an initial state that will lead to the postcondition being satisfied.

In [8]:
from tinyscript_util import state_from_z3_model

s = z3.Solver()
s.add(box(swap_alpha, fmla_enc(fmla_parse("x == 1 && y == 0"))))
s.check()
state = state_from_z3_model(swap_alpha, s.model())
state

State(variables={'x': 0, 'y': 1})

And we can run the interpreter on this state to check that the postcondition is satisfied.

In [9]:
from interpreter import exc

exc(state, swap_alpha)

(State(variables={'x': 1, 'y': 0}), <Status.Terminated: 1>, None)

## A simple contract checker

Using `box`, we can also implement a (very) simple contract verifier.
The function below returns `True` if the program satisfies its contract, and if it doesn't, then it returns a counterexample input that will violate the contract.

Note the way that it works: `z3` is a *satisfiability* solver, not a validity checker. We've already seen that these semantic notions are duals of eachother, so to check for the validity of `P -> [alpha] Q` using `z3`, we negate and check for satisfiability. This negated formula is called a *verification condition*. It is unsatisfiable iff the program follows its contract, and otherwise any satisfying assignments correspond to an initial state (i.e., set of inputs) that will lead the program to violate its contract.

This type of verification is called *bounded model checking*.

In [10]:
from symbolic import Result
from tinyscript_util import (
    check_sat,
    state_from_z3_model
)

# verify contracts of the form P --> [alpha] Q
def verify_contract(
    alpha: tn.Program, 
    P: tn.Formula, Q: tn.Formula
) -> tuple[Result, tn.State]:
    weakest_pre = box(alpha, Q)
    res, model = check_sat([Not(Implies(P, weakest_pre))])
    if res == unsat:
        return (Result.Satisfies, None)
    elif res == sat:
        state = state_from_z3_model(alpha, model)
        return (Result.Violates, state)
    else:
        return (Result.Unknown, None)

And we can check the contract from earlier

In [11]:
verify_contract(swap_alpha, pre, post)

(<Result.Satisfies: 1>, None)

If we give it a buggy program, it will return a counterexample input

In [12]:
buggy = "x := x + y; y := x - y; x := x + y"
buggy_alpha = parse(buggy)
status, state = verify_contract(buggy_alpha, pre, post)
status, state

(<Result.Violates: 2>, State(variables={'x': 1, 'y': 0}))

Running the interpreter on this input will show that the postcondition is violated

In [13]:
exc(state, buggy_alpha)

(State(variables={'x': 2, 'y': 1}), <Status.Terminated: 1>, None)

## Checking Invariant Properties

Below is a sample checker that follows the analysis workflow outlined in the lab handout. It targets invariant properties, which are characterized by a formula $P$ that must remain true at all times throughout a program's execution.

$$
\Phi_P = \{\sigma : \forall i. 0\le i < |\sigma| \rightarrow \sigma_i \vDash P\}
$$

In practice, the above definition is too strict, because programs have no control over their initial state. For example, consider the invariant property which says that `x`, `y`, and `z` should all remain non-negative. The following program begins by attempting to establish the invariant.
```
x := 0;
y := 0;
z := 0;
...
```
However, it does not satisfy the invariant property, because it may begin in an initial state that maps `x`, `y`, and `z` to negative numbers, and isn't able to correct it until entering the fourth state of a trace. To address this, we will give the program a grace period in which it must establish the invariant. After it has done so, then the invariant must remain true for the remainder of the trace.

$$
\Phi_P = \{\sigma : \exists i. \sigma_i \vDash P \land \forall j. i \le j < |\sigma| \rightarrow \sigma_i \vDash P\}
$$

To enforce this property, we will add instrumentation to each instruction that could potentially either establish or violate the invariant. For the live coding exercise, we only consider assignment, composition, conditional, and while statements in our program. Among these, the only type of instruction that can affect the invariant is assignment. However, if we were considering the full tinyscript language, then we would need to think about whether `output` commands could be relevant, and how to deal with the effects of `abort`.

The instrumentation that we add will track two policy variables, `#inv_established` and `#inv_true`. `#inv_established` is initialized by a conditional statement which checks whether the initial state satisfies `P`. After this initialization, none of the instrumentation will ever set `#inv_established` to 0, reflecting that having established the invariant cannot be undone.

For each assignment instruction $\alpha$, we will determine whether $[\alpha]\,P \leftrightarrow P$. If so, then there is no need to add instrumentation. If not, then we must set the policy variables accordingly. The instrumentation constructs a box-free equivalent of $[\alpha]\,P$, and add a conditional statement that branches on this formula: the "then" branch handles the case where the invariant will be true after the instrumented instruction, and the "else" branch where it will be false.
* In the "then" branch, if `#inv_established` is currently 0, then we want to set both policy variables to 1. Add the corresponding conditional statement to construct the body of this branch.
* In the "else" branch, the instrumentation sets `#inv_true` to 0.

As we will see below, there are a few optimizations for this approach that are easy to implement, but this instrumentation is sufficient to establish the desired correspondence. If $\alpha'$ is the instrumented version of $\alpha$, then:

$$
\alpha~\text{satisfies the invariant policy for formula}~P
\quad\Longleftrightarrow\quad
\vDash [\alpha'](\mathtt{\#inv\_established} = 1 \land \mathtt{\#inv\_true} = 1)
$$

Having this, `symbolic_check` is straightforward to implement

### Implementation

We will start by writing some utility functions. When we compute the box-free equivalent of $[\alpha] P$ using `box`, the result will be a Z3 `BoolRef`. But to construct instrumentation, we need the corresponding formula as a `tn.Formula`. The utility `z3_to_fmla` below accomplishes this.

In [14]:
def z3_to_fmla(P: BoolRef):
    """
    Convert a Z3 BoolRef to a tinyscript formula
    """
    if is_int_value(P):
        return tn.Const(P.as_long())
    elif is_const(P) and not is_true(P) and not is_false(P):
        return tn.Var(str(P))
    elif is_add(P):
        return tn.Sum(z3_to_fmla(P.children()[0]),
                      z3_to_fmla(P.children()[1]))
    elif is_sub(P):
        return tn.Difference(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1]))
    elif is_mul(P):
        return tn.Product(z3_to_fmla(P.children()[0]),
                          z3_to_fmla(P.children()[1]))
    elif is_true(P):
        return tn.TrueC()
    elif is_false(P):
        return tn.FalseC()
    elif is_lt(P):
        return tn.LtF(z3_to_fmla(P.children()[0]), 
                      z3_to_fmla(P.children()[1]))
    elif is_le(P):
        return tn.OrF(tn.LtF(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1])),
                      tn.EqF(z3_to_fmla(P.children()[0]),
                             z3_to_fmla(P.children()[1])))
    elif is_gt(P):
        return tn.NotF(tn.LtF(z3_to_fmla(P.children()[0]), 
                              z3_to_fmla(P.children()[1])))
    elif is_eq(P):
        return tn.EqF(z3_to_fmla(P.children()[0]), 
                      z3_to_fmla(P.children()[1]))
    elif is_not(P):
        return tn.NotF(z3_to_fmla(P.children()[0]))
    elif is_and(P):
        return tn.AndF(z3_to_fmla(P.children()[0]),
                       z3_to_fmla(P.children()[1]))
    elif is_or(P):
        return tn.OrF(z3_to_fmla(P.children()[0]),
                       z3_to_fmla(P.children()[1]))
    elif is_implies(P):
        return tn.ImpliesF(z3_to_fmla(P.children()[0]),
                           z3_to_fmla(P.children()[1]))
    else: 
        raise TypeError(
            f"Expected BoolRef, got {P}"
        )

We will also be checking formulas for equivalence when adding instrumentation. The utility `fmlas_equiv` does so.

In [15]:
def fmlas_equiv(P: BoolRef, Q: BoolRef) -> bool:
    """
    Test whether P and Q are equivalent, i.e., whether |= P <--> Q
    """
    res, _ = check_sat([Not(P == Q)])
    if res == sat or res == unknown:
        # If unknown, we conservatively assume that the
        # formulas are not equivalent. This will not break
        # anything, but merely result in a potentially
        # unnecessary instrumentation.
        return False
    else:
        return True

We'll define globals for the policy variables, rather than hard-coding them throughout our implementation.

In [16]:
SETUP_VAR = '#inv_established'
INV_VAR = '#inv_true'

Now for the actual instrumentation. The function `invariant_instrument` constructs the instrumentation statements to place before each assignment. It takes a single `BoolRef` argument, which should be the box-free equivalent of $[\alpha]\,P$, where $\alpha$ is the assignment being instrumented. It implements the approach described earlier, except with two optimizations.
* If the argument `Q` is the constant `False`, then it means that the invariant will certainly be violated when the assignment is executed. In this case, the instrumentation just sets `#inv_true` to 0.
* Similarly, if the argument `Q` is the constant `True`, then the invariant formula `P` will certainly be true after then assignment is executed. In this case, the instrumentation just sets the policy variables to 1 (if appropriate), and does not contain a conditional to test if `Q` is true.

In [17]:
def invariant_instrument(Q: BoolRef) -> tn.Prog:
    """
    Construct instrumentation to enforce an invariant P,
    to be placed immediately before an assignment alpha.
    
    Args:
        Q (z3.BoolRef): A box-free formula that is equivalent
            to [alpha] P.
    
    Returns:
        tn.Prog: A tinyscript program that will set the
            policy variables appropriately to enforce the
            invariant P.
    """
    true_ins = tn.If(tn.EqF(tn.Var(SETUP_VAR), tn.Const(0)),
                     tn.Seq(tn.Asgn(INV_VAR, tn.Const(1)),
                            tn.Asgn(SETUP_VAR, tn.Const(1))),
                     tn.Skip())
    false_ins = tn.Asgn(INV_VAR, tn.Const(0))
    if is_true(Q):
        return true_ins
    elif is_false(Q):
        return false_ins
    else:
        return tn.If(z3_to_fmla(Q),
                     true_ins,
                     false_ins)

Putting this all together, we can write `add_instrumentation`, which recurses on the structure of a program to add policy-state maintenance instrumentation before each assignment.

In [18]:
def add_instrumentation(alpha: tn.Prog, inv: tn.Formula) -> tn.Prog:
    """
    Construct instrumentation to enforce an invariant P,
    to be placed immediately before an assignment alpha.
    
    Args:
        alpha (tn.Prog): The program to instrument
        inv (tn.Formula): The invariant formula to enforce
    
    Returns:
        tn.Prog: A tinyscript program with instrumentation before each
            assignment.
            
    Raises:
        TypeError: The provided alpha is not a valid tinyscript program.
    """
    match alpha:
        # assignments can violate the invariant, so instrument them directly
        case tn.Asgn(name, aexp):
            pre = box(alpha, fmla_enc(inv))
            # pre will be equivalent to inv if and only if the assignment
            # has no effect on whether the invariant will be violated or
            # established, so we don't add instrumentation if this is
            # the case.
            if not fmlas_equiv(fmla_enc(inv), pre):
                ins = invariant_instrument(pre)
                if ins != tn.Skip():
                    return tn.Seq(ins, alpha)
            return alpha
        # composition cannot violate the invariant unless through either
        # of its constituents, so recurse and do not add instrumentation directly
        case tn.Seq(alpha_p, beta_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            ins_beta = add_instrumentation(beta_p, inv)
            return tn.Seq(ins_alpha, ins_beta)
        # same with conditionals
        case tn.If(p, alpha_p, beta_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            ins_beta = add_instrumentation(beta_p, inv)
            return tn.If(p, ins_alpha, ins_beta)
        # same with while loops
        case tn.While(q, alpha_p):
            ins_alpha = add_instrumentation(alpha_p, inv)
            return tn.While(q, ins_alpha)
        # skips do nothing for invariants, so no instrumentation
        case tn.Skip():
            return alpha
        case _:
            raise TypeError(
                f"instrument got {type(alpha)} ({alpha}), not Prog"
            )

def instrument(alpha: tn.Prog, invariant: tn.Formula) -> tn.Prog:
    instr = add_instrumentation(alpha, invariant)
    initialize = tn.If(invariant,
                       tn.Seq(tn.Asgn(SETUP_VAR, tn.Const(1)),
                              tn.Asgn(INV_VAR, tn.Const(1))),
                       tn.Seq(tn.Asgn(SETUP_VAR, tn.Const(0)),
                              tn.Asgn(INV_VAR, tn.Const(0))))
    return tn.Seq(initialize, instr)

Let's test it out. Note that there isn't a great invariant for this program, so we'll just use `y < x`.

In [19]:
inv = fmla_parse("y < x")
instrumented = instrument(swap_alpha, inv)
print(stringify(instrumented))

if ((y)<(x)) then
    #inv_established := 1;
    #inv_true := 1
else
    #inv_established := 0;
    #inv_true := 0
endif;
if (!((((x)+(y))<(y))||(((x)+(y))==(y)))) then
    if ((#inv_established)==(0)) then
        #inv_true := 1;
        #inv_established := 1
    else
        skip
    endif
else
    #inv_true := 0
endif;
x := (x)+(y);
if (!(((x)<((x)+((-1)*(y))))||((x)==((x)+((-1)*(y)))))) then
    if ((#inv_established)==(0)) then
        #inv_true := 1;
        #inv_established := 1
    else
        skip
    endif
else
    #inv_true := 0
endif;
y := (x)-(y);
if (!((((x)+((-1)*(y)))<(y))||(((x)+((-1)*(y)))==(y)))) then
    if ((#inv_established)==(0)) then
        #inv_true := 1;
        #inv_established := 1
    else
        skip
    endif
else
    #inv_true := 0
endif;
x := (x)-(y)


Now we implement `symbolic_check`. If the solver returns `sat`, then `symbolic_check` will attempt to generate an initial state that will lead the program to violate the invariant. If it sees that the invariant is violated in the final state (i.e., `#inv_true` is 0), then it returns `Result.Violates`. Otherwise it returns `Result.Unknown`.

In [20]:
from interpreter import exc
from symbolic import Result

def symbolic_check(alpha: tn.Prog, P: tn.Formula, timeout=10) -> Result:
    alpha_p = instrument(alpha, P)
    post = tn.AndF(tn.EqF(tn.Var(SETUP_VAR), tn.Const(1)),
                   tn.EqF(tn.Var(INV_VAR), tn.Const(1)))
    weakest_pre = box(alpha, fmla_enc(post))
    
    res, model = check_sat([Not(weakest_pre)], timeout=timeout)
    
    if res == unsat:
        return Result.Satisfies
    elif res == sat:
        state = state_from_z3_model(alpha, model, complete=True)
        final_state = exc(state, alpha_p, max_steps=1.e6, quiet=False)
        if final_state[0].variables[INV_VAR] == 0:
            return Result.Violates
    return Result.Unknown

We see that the checker is able to find a counterexample. Most likely, corresponding to an initial state like `x = 0, y = 1`.

In [21]:
symbolic_check(swap_alpha, inv)

<Result.Violates: 2>